Wahlpflichtfach Künstliche Intelligenz I: Praktikum 

---

# 10 - Scikit-learn - Modellauswahl

Unter Modellauswahl ist die Auswahl des ML-Modells zu verstehen, dass die gegebenen Daten bestmöglich erklärt. Da es aber eine Vielzahl an unterschiedlichen Modellen gibt, die wiederum große Parameterbereiche haben, ist diese Aufgabe eine sehr kritische. Wenn zum Beispiel ein Modell ausgewählt wird, das zwar sehr gut auf den Trainingsdaten performt hat, aber auf neuen Daten kläglich versagt, hat dieses die zugrundeliegende Datenverteilung nicht gelernt. 

In diesem Notebook werden wir auf die folgenden Themen wingehen:
- Kreuzvalidierung (Cross-validation)
- Hyperparametertuning
- Metriken und Ergebnisse
- Visualisierung von Ergebnissen

Das Thema Modelauswahl und -evaluierung wird auch sehr ausführlich im [sklearn-Benutzerhandbuch](https://scikit-learn.org/stable/model_selection.html) behandelt.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import datasets

## Kreuzvalidierung
Das Lernen der Parameter einer Vorhersagefunktion und das Testen auf denselben Daten ist ein methodischer Fehler: Ein Modell, das einfach nur die Labels der Stichproben, die es gerade gesehen hat, wiederholen würde, hätte ein perfektes Ergebnis, würde für ungesehenen Daten aber nichts Brauchbares vorhersagen können. Diese Situation wird als Overfitting bezeichnet. Um dies zu vermeiden, ist es bei der Durchführung eines (überwachten) Experiments zum maschinellen Lernen üblich, einen Teil der verfügbaren Daten als Testset vorzuhalten.

### Einfaches Aufteilen der Daten
Um die Daten einfach nur Aufzuteilen, kann die [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)-Methode verwended werden.

In [ ]:
from sklearn.model_selection import train_test_split

# load data
X, y = datasets.load_iris(return_X_y=True)
print(X.shape, y.shape)

# split the data into 2 sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

Nun kann die Support-Vektor-Maschine mit den Trainingsdaten trainiert werden und mit den Testdaten evaluiert werden.

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='linear', C=0.001).fit(X_train, y_train)
clf.score(X_test, y_test)

Ausprobieren von anderen Werten für C.

In [ ]:
clf = svm.SVC(kernel='linear', C=0.01).fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
clf = svm.SVC(kernel='linear', C=0.1).fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
clf.score(X_test, y_test)

### Warum Kreuzvalidierung?
Bei der Auswertung verschiedener Einstellungen ("Hyperparameter") für Vorhersagefunktionen, wie z. B. des C-Werts, die für eine SVM manuell gesetzt werden muss, besteht immer noch die Gefahr des Overfittings auf dem Testset, da die Parameter so lange verändert werden können, bis die Vorhersagefunktionen optimal arbeitet. Auf diese Weise kann das Wissen über das Testset in das Modell "durchsickern" und die Auswertungsmetriken berichten nicht mehr über die Generalisierungsleistung. Um dieses Problem zu lösen, kann ein weiterer Teil des Datensatzes als sogenanntes "Validierungsset" vorgehalten werden: Das Training erfolgt auf dem Trainingsset, danach wird die Auswertung auf dem Validierungsset durchgeführt, und wenn das Experiment erfolgreich zu sein scheint, kann die abschließende Auswertung auf dem Testset erfolgen.

Durch die Aufteilung der verfügbaren Daten in drei Sätze wird jedoch die Anzahl der Datenpunkte, die für das Lernen des Modells verwendet werden können, drastisch reduziert. Außerdem können die Ergebnisse von einer bestimmten zufälligen Wahl für das Paar von (Trainings-, Validierungs-)Sets abhängen.

Um diese Probleme zu überkommen kann das Kreuzvalidierungsverfahren angewandt werden. Bei diesem Verfahren wird immer noch ein Testset benötigt, aber das Validierungsset ist nicht länger nötig. In der Basisversion wird das Trainingsset während des Trainings in $k$ Sets unterteilt. Anschließend wird der Algorithmus $k$ mal auf $k-1$ Sets trainiert. Die Evaluierung des Modells erfolgt dabei auf dem $k$ten Set.

<img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png">

### Verwendung von Kreuzvalidierung
Der einfachste Weg Kreuzvalidierung zu verwenden ist die [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score)-Hilfsfunktion.

In [ ]:
from sklearn.model_selection import cross_val_score
clf = svm.SVC(kernel='linear', C=1, random_state=42)

k = 5
scores = cross_val_score(clf, X, y, cv=k)
scores

Die Funktion gibt ein Array der Länge $k$ zurück. Jedes Element ist der Score für das jeweilige Teilset.

In [ ]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Standardmäßig wird als Ergebniss jeder Kreuzvalidierungsiteration der Wert der `score`-Methode des Modells zurückgegeben. Um dies zu ändern, kann der `scoring` Parameter gesetzt werden.

In [ ]:
scores = cross_val_score(clf, X, y, cv=5, scoring='f1_macro')
scores

Einen weiteren Weg Kreuzvalidierung durchzuführen bietet die [cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html)-Methode. Dieser können, im Gegensatz zur `cross_val_score`-Funktion, mehrere Metriken übergeben werden. Außerdem gibt die Methode zusätzlich zu den Ergebnissen auch Informationen über Laufzeiten zurück. 

In [ ]:
from sklearn.model_selection import cross_validate
scoring = ['precision_macro', 'recall_macro']
clf = svm.SVC(kernel='linear', C=1, random_state=0)
scores = cross_validate(clf, X, y, scoring=scoring)
scores

### Kreuzvalidierungsiteratoren
An beide Funktionen kann mit Hilfe des Parameters `cv` entweder ein Integer-Wert $k$ oder ein Kreuzvalidierungsiteratoren übergeben werden. Ersteres resultiert darin, dass der [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) bzw. der [StratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html)-Iterator mit $k$ folds verwendet wird. Im Zweiten fall wird der entsprechende [Kreuzvalidierungsiterator](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators) verwendet, um das Datenset während des Trainings aufzuteilen.

## Hyperparametertuning
Hyperparameter sind all die Parameter, die ein ML-Algorithmus nicht selber lernt. Zum Beispiel ist der `C`-Parameter eines SVMs ein Hyperparameter, den wir vorhin schon einmal manuell optimiert haben. Damit nicht jeder Parameter oder besser gesagt jede Parameterkombination manuell optimiert werden muss, bietet sklearn verschiedene Algorithmen dafür an. Diese Algorithmen suchen im Hyperparameterraum die besten Parameterkombinationen für den ML-Algorithmus. 

### Umfassende Rastersuche (Grid Search)
Die von [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) bereitgestellte Rastersuche generiert umfassend Kandidaten aus einem mit dem Parameter `param_grid` angegebenen Raster von Parameterwerten. Zum Beispiel gibt der folgende `param_grid`

In [ ]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]

an, dass zwei Raster untersucht werden sollen: eines mit einem linearen Kernel und C-Werten in [1, 10, 100, 1000] und das zweite mit einem RBF-Kernel und dem Kreuzprodukt aus C-Werten im Bereich [1, 10, 100, 1000] und Gamma-Werten in [0,001, 0,0001].

In [ ]:
from sklearn.model_selection import GridSearchCV

# create a SVC and optimize it with a grid search
svc = svm.SVC()
clf = GridSearchCV(svc, param_grid)
clf.fit(X, y)

# get the best estimator
clf.best_estimator_

Anzeigen aller Ergebnisse der Kreuzvalidierung

In [ ]:
clf.cv_results_

### Zufällige Rastersuche
Anstatt die Parameter fest vorzuschreiben können diese auch zufällig aus entsprechenden Verteilungen gezogen werden. Dafür kann die Klasse [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) verwendet werden. Ein beispielhaftes Grid könnte dort wie folgt aussehen.

In [ ]:
import scipy

param_grid = {'C': scipy.stats.expon(scale=100), 'gamma': scipy.stats.expon(scale=.1),
  'kernel': ['rbf'], 'class_weight':['balanced', None]}

### Schnellere Rastersuche
Da unter Umständen aus sehr vielen Parametern die besten gefunden werden müssen, gibt es für beide vorherige Varainten auch eine schneller Implementierung. Dabei werden zuerst alle Parameterkombinationen auf einem Bruchteil der Daten trainiert und evaluiert. Anschließend werden die 50% der Parameterkombinationen entfernt, die am schlechtesten abgeschnitten haben. Dann wird die Datenmenge ein wenig erhöht und die nächste Runde beginnt. 

<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_successive_halving_iterations_001.png">

Für eine schnelle, umfassende Suche kann die Klasse [HalvingGridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) verwendet werden und für eine schnelle, zufällige Suche die Klasse [HalvingRandomSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html).

### Rastersuche und Pipelines
Selbstverständlich kann auch ein Pipeline-Objekt an die Rastersuche übergeben werden. Dafür müssen die Einträge im Parametergrid dann wieder das Format `<estimator>__<parameter>` haben.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_moons

# create some data
X, y = make_moons()

# setup the pipeline
calibrated_forest = CalibratedClassifierCV(estimator=RandomForestClassifier(n_estimators=10))
pipe = Pipeline([('select', SelectKBest()), ('model', calibrated_forest)])

# define the parameter grid
param_grid = {
   'select__k': [1, 2],
   'model__estimator__max_depth': [2, 4, 6, 8]
}

# fit the best parameters for the whole pipeline
search = GridSearchCV(pipe, param_grid, cv=5).fit(X, y)
search.best_estimator_

## Metriken
Metriken werden zur Evaluation/Validierung von ML-Modellen verwendet. Die folgende Tabelle gibt ein Überblick über die meistverwendeten Scoring-Funktionen. Um beispielsweise in der Kreuzvalidierung eine andere Funktion als die Standardfunktion des ML-Algorithmuses zu verwenden, kann zum Beispiel an das `GridSearchCV`-Objekt einen Wert aus der Spalte _Scoring_ an den Parameter `scoring`übergeben werden. Für Scores gilt generell, je höher der Wert, desto besser. Daher gibt es für Metriken, die zum Beispiel Distanzen messen, negierte Metriken (`neg_mean_squared_error`).

<table class="docutils align-default">
<colgroup>
<col style="width: 31%">
<col style="width: 40%">
<col style="width: 29%">
</colgroup>
<thead>
<tr class="row-odd"><th class="head"><p>Scoring</p></th>
<th class="head"><p>Function</p></th>
<th class="head"><p>Comment</p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p><strong>Classification</strong></p></td>
<td></td>
<td></td>
</tr>
<tr class="row-odd"><td><p>‘accuracy’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score" title="sklearn.metrics.accuracy_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.accuracy_score</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-even"><td><p>‘balanced_accuracy’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html#sklearn.metrics.balanced_accuracy_score" title="sklearn.metrics.balanced_accuracy_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.balanced_accuracy_score</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-odd"><td><p>‘top_k_accuracy’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.top_k_accuracy_score.html#sklearn.metrics.top_k_accuracy_score" title="sklearn.metrics.top_k_accuracy_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.top_k_accuracy_score</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-even"><td><p>‘average_precision’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html#sklearn.metrics.average_precision_score" title="sklearn.metrics.average_precision_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.average_precision_score</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-odd"><td><p>‘neg_brier_score’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.brier_score_loss.html#sklearn.metrics.brier_score_loss" title="sklearn.metrics.brier_score_loss"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.brier_score_loss</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-even"><td><p>‘f1’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score" title="sklearn.metrics.f1_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.f1_score</span></code></a></p></td>
<td><p>for binary targets</p></td>
</tr>
<tr class="row-odd"><td><p>‘f1_micro’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score" title="sklearn.metrics.f1_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.f1_score</span></code></a></p></td>
<td><p>micro-averaged</p></td>
</tr>
<tr class="row-even"><td><p>‘f1_macro’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score" title="sklearn.metrics.f1_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.f1_score</span></code></a></p></td>
<td><p>macro-averaged</p></td>
</tr>
<tr class="row-odd"><td><p>‘f1_weighted’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score" title="sklearn.metrics.f1_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.f1_score</span></code></a></p></td>
<td><p>weighted average</p></td>
</tr>
<tr class="row-even"><td><p>‘f1_samples’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score" title="sklearn.metrics.f1_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.f1_score</span></code></a></p></td>
<td><p>by multilabel sample</p></td>
</tr>
<tr class="row-odd"><td><p>‘neg_log_loss’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html#sklearn.metrics.log_loss" title="sklearn.metrics.log_loss"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.log_loss</span></code></a></p></td>
<td><p>requires <code class="docutils literal notranslate"><span class="pre">predict_proba</span></code> support</p></td>
</tr>
<tr class="row-even"><td><p>‘precision’ etc.</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score" title="sklearn.metrics.precision_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.precision_score</span></code></a></p></td>
<td><p>suffixes apply as with ‘f1’</p></td>
</tr>
<tr class="row-odd"><td><p>‘recall’ etc.</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score" title="sklearn.metrics.recall_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.recall_score</span></code></a></p></td>
<td><p>suffixes apply as with ‘f1’</p></td>
</tr>
<tr class="row-even"><td><p>‘jaccard’ etc.</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.jaccard_score.html#sklearn.metrics.jaccard_score" title="sklearn.metrics.jaccard_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.jaccard_score</span></code></a></p></td>
<td><p>suffixes apply as with ‘f1’</p></td>
</tr>
<tr class="row-odd"><td><p>‘roc_auc’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score" title="sklearn.metrics.roc_auc_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.roc_auc_score</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-even"><td><p>‘roc_auc_ovr’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score" title="sklearn.metrics.roc_auc_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.roc_auc_score</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-odd"><td><p>‘roc_auc_ovo’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score" title="sklearn.metrics.roc_auc_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.roc_auc_score</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-even"><td><p>‘roc_auc_ovr_weighted’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score" title="sklearn.metrics.roc_auc_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.roc_auc_score</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-odd"><td><p>‘roc_auc_ovo_weighted’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score" title="sklearn.metrics.roc_auc_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.roc_auc_score</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-even"><td><p><strong>Clustering</strong></p></td>
<td></td>
<td></td>
</tr>
<tr class="row-odd"><td><p>‘adjusted_mutual_info_score’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_mutual_info_score.html#sklearn.metrics.adjusted_mutual_info_score" title="sklearn.metrics.adjusted_mutual_info_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.adjusted_mutual_info_score</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-even"><td><p>‘adjusted_rand_score’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html#sklearn.metrics.adjusted_rand_score" title="sklearn.metrics.adjusted_rand_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.adjusted_rand_score</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-odd"><td><p>‘completeness_score’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.completeness_score.html#sklearn.metrics.completeness_score" title="sklearn.metrics.completeness_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.completeness_score</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-even"><td><p>‘fowlkes_mallows_score’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.fowlkes_mallows_score.html#sklearn.metrics.fowlkes_mallows_score" title="sklearn.metrics.fowlkes_mallows_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.fowlkes_mallows_score</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-odd"><td><p>‘homogeneity_score’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_score.html#sklearn.metrics.homogeneity_score" title="sklearn.metrics.homogeneity_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.homogeneity_score</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-even"><td><p>‘mutual_info_score’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mutual_info_score.html#sklearn.metrics.mutual_info_score" title="sklearn.metrics.mutual_info_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.mutual_info_score</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-odd"><td><p>‘normalized_mutual_info_score’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.normalized_mutual_info_score.html#sklearn.metrics.normalized_mutual_info_score" title="sklearn.metrics.normalized_mutual_info_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.normalized_mutual_info_score</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-even"><td><p>‘rand_score’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.rand_score.html#sklearn.metrics.rand_score" title="sklearn.metrics.rand_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.rand_score</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-odd"><td><p>‘v_measure_score’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.v_measure_score.html#sklearn.metrics.v_measure_score" title="sklearn.metrics.v_measure_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.v_measure_score</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-even"><td><p><strong>Regression</strong></p></td>
<td></td>
<td></td>
</tr>
<tr class="row-odd"><td><p>‘explained_variance’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.explained_variance_score.html#sklearn.metrics.explained_variance_score" title="sklearn.metrics.explained_variance_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.explained_variance_score</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-even"><td><p>‘max_error’</p></td>
<td><p><a class="reference internal" href="generated/sklearn.metrics.max_error.html#sklearn.metrics.max_error" title="https://scikit-learn.org/stable/modules/sklearn.metrics.max_error"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.max_error</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-odd"><td><p>‘neg_mean_absolute_error’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html#sklearn.metrics.mean_absolute_error" title="sklearn.metrics.mean_absolute_error"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.mean_absolute_error</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-even"><td><p>‘neg_mean_squared_error’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html#sklearn.metrics.mean_squared_error" title="sklearn.metrics.mean_squared_error"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.mean_squared_error</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-odd"><td><p>‘neg_root_mean_squared_error’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html#sklearn.metrics.mean_squared_error" title="sklearn.metrics.mean_squared_error"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.mean_squared_error</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-even"><td><p>‘neg_mean_squared_log_error’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_log_error.html#sklearn.metrics.mean_squared_log_error" title="sklearn.metrics.mean_squared_log_error"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.mean_squared_log_error</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-odd"><td><p>‘neg_median_absolute_error’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.median_absolute_error.html#sklearn.metrics.median_absolute_error" title="sklearn.metrics.median_absolute_error"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.median_absolute_error</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-even"><td><p>‘r2’</p></td>
<td><p><a class="reference internal" href="generated/sklearn.metrics.r2_score.html#sklearn.metrics.r2_score" title="sklearn.metrics.r2_score"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.r2_score</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-odd"><td><p>‘neg_mean_poisson_deviance’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_poisson_deviance.html#sklearn.metrics.mean_poisson_deviance" title="sklearn.metrics.mean_poisson_deviance"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.mean_poisson_deviance</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-even"><td><p>‘neg_mean_gamma_deviance’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_gamma_deviance.html#sklearn.metrics.mean_gamma_deviance" title="sklearn.metrics.mean_gamma_deviance"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.mean_gamma_deviance</span></code></a></p></td>
<td></td>
</tr>
<tr class="row-odd"><td><p>‘neg_mean_absolute_percentage_error’</p></td>
<td><p><a class="reference internal" href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_percentage_error.html#sklearn.metrics.mean_absolute_percentage_error" title="sklearn.metrics.mean_absolute_percentage_error"><code class="xref py py-func docutils literal notranslate"><span class="pre">metrics.mean_absolute_percentage_error</span></code></a></p></td>
<td></td>
</tr>
</tbody>
</table>

### Accuracy
Die Accuracy gibt den Anteil der korrekt klassifizierten Datenpunkte zurück.

$$ accuracy(y, \hat{y})= \frac{1}{n_{samples}}\sum_{i=0}^{n_{samples}-1}1(\hat{y}_i = y_i) $$

where $1(x)$ is the indicator function.

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = [0, 2, 1, 3]
y_true = [0, 1, 2, 3]
accuracy_score(y_true, y_pred)

Wenn wir nicht normalisieren, bekommen wir die Anzahl an korrekt klassifizierten Datenpunkten zurück.

In [ ]:
accuracy_score(y_true, y_pred, normalize=False)

Wenn ein Datenpunkt zu mehreren Klassen gehört, müssen alle Klassifizierungen übereinstimmen.

In [ ]:
accuracy_score(np.array([[0, 1], [1, 1]]), np.ones((2, 2)))

### Balanced Accuracy
Die Balanced Accuracy berechnet einen Accuracy-Wert pro Klasse. Dadurch ist sie besser für unausgeglichene Datensets geeignet.

$$ balanced\_accuracy=\frac{1}{2}(\frac{TP}{TP+FN}+\frac{TN}{TN+FP})$$

In [ ]:
from sklearn.metrics import balanced_accuracy_score
y_pred = [0, 0, 0, 0]
y_true = [0, 0, 0, 1]
balanced_accuracy_score(y_true, y_pred)

### Confusion Matrix
In sklearn steht jede Zeile für das wirkliche Label und jede Spalte für das vorhergesagte Label. 

In [ ]:
from sklearn.metrics import confusion_matrix
y_true = [2, 0, 2, 2, 0, 1]
y_pred = [0, 0, 2, 2, 0, 2]
confusion_matrix(y_true, y_pred)

Mit Hilfe der [plot_confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html)-Funktion kann die Matrix visualisiert werden.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

# import and split the data
iris = datasets.load_iris()
X = iris.data
y = iris.target
class_names = iris.target_names
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# define the classifier and fit it
classifier = svm.SVC(kernel='linear', C=0.01).fit(X_train, y_train)

# create the plot
fig, ax = plt.subplots(1, figsize=(10, 8))
_ = ConfusionMatrixDisplay.from_estimator(
    classifier, 
    X_test, 
    y_test, 
    display_labels=iris.target_names, 
    cmap=plt.cm.Blues,
    ax=ax)

### Klassifikationsbericht
Mit der [classification_report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)-Funktion ist es möglich sich ein Textbericht, der die wichtigsten Klassifikationsmetriken beinhaltet, zu erzeugen.

In [ ]:
from sklearn.metrics import classification_report
y_true = [0, 1, 2, 2, 0]
y_pred = [0, 0, 2, 1, 0]
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_true, y_pred, target_names=target_names))

Weitere Informationen zu den Metriken finden Sie [hier](https://scikit-learn.org/stable/modules/model_evaluation.html#).

## Visualisierung von Ergebnissen
Zur [Visualisierung von Ergebnissen](https://scikit-learn.org/stable/modules/learning_curve.html#) bietet sklearn zwei Helfermethoden. 

### Validtion Curve

Die [validation_curve](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.validation_curve.html)-Methode hilft dabei die Tranings- und Validierungsergebnisse unter Einfluss eines Hyperparameters zu visualisieren.

In [ ]:
from sklearn.model_selection import validation_curve

# load and shuffel the data
X, y = datasets.load_iris(return_X_y=True)
indices = np.arange(y.shape[0])
np.random.shuffle(indices)
X, y = X[indices], y[indices]

# collect data for plotting
c_values = [0.5, 1, 2, 3, 4, 5]
train_scores, valid_scores = validation_curve(svm.SVC(kernel='linear'), X, y, 
    param_name='C', param_range=c_values, cv=5)

In [ ]:
train_scores

In [ ]:
valid_scores

Mit diesen Daten können wir jetzt einen Graphen erstellen.

In [ ]:
# first, calculate the means and standard deviations
train_scores_mean = train_scores.mean(axis=1)
train_scores_std = train_scores.std(axis=1)
valid_scores_mean = valid_scores.mean(axis=1)
valid_scores_std = valid_scores.std(axis=1)

# create the figure
plt.figure(figsize=(10, 10))
plt.title("Validation Curve of SVM with different C values")
plt.xlabel("C")
plt.ylabel("Score")
# plot the training score
plt.plot(c_values, train_scores_mean, label="Training score", color="darkorange")
plt.fill_between(c_values, 
                 train_scores_mean - train_scores_std, 
                 train_scores_mean + train_scores_std, 
                 color="darkorange",
                 alpha=0.2
                )
# plot the validation score
plt.plot(c_values, valid_scores_mean, label="Cross-validation score", color="navy")
plt.fill_between(c_values, 
                 valid_scores_mean - valid_scores_std, 
                 valid_scores_mean + valid_scores_std, 
                 color="navy",
                 alpha=0.2
                )

plt.legend(loc="best")
plt.show()

### Learning Curve
Mit Hilfe der [learing_curve]()-Methode können wir zeigen wie der Validierungs- und Trainingscore sich verändern, wenn unterschiedliche Trainingsdatenmengen verwendet werden.

In [ ]:
from sklearn.model_selection import learning_curve

# collect the data
train_sizes, train_scores, valid_scores = learning_curve(
    svm.SVC(kernel='linear'), X, y, train_sizes=np.arange(24, 121, 12), cv=5)
train_sizes

In [ ]:
train_scores

In [ ]:
valid_scores

In [ ]:
# first, calculate the means and standard deviations
train_scores_mean = train_scores.mean(axis=1)
train_scores_std = train_scores.std(axis=1)
valid_scores_mean = valid_scores.mean(axis=1)
valid_scores_std = valid_scores.std(axis=1)

# create the figure
plt.figure(figsize=(10, 10))
plt.title("Learning  Curve")
plt.xlabel("Training examples")
plt.ylabel("Score")
# plot the training score
plt.plot(train_sizes, train_scores_mean, label="Training score", color="darkorange")
plt.fill_between(train_sizes, 
                 train_scores_mean - train_scores_std, 
                 train_scores_mean + train_scores_std, 
                 color="darkorange",
                 alpha=0.2
                )
# plot the validation score
plt.plot(train_sizes, valid_scores_mean, label="Cross-validation score", color="navy")
plt.fill_between(train_sizes, 
                 valid_scores_mean - valid_scores_std, 
                 valid_scores_mean + valid_scores_std, 
                 color="navy",
                 alpha=0.2
                )

plt.legend(loc="best")
plt.show()

---

Wahlpflichtach Künstliche Intelligenz I: Praktikum